In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import covid_api
from config import google_api
import gmaps


In [3]:
# -----------------------
# Get risk factor data for North Carolina, based on census data
# -----------------------

# Import CRE csv from
# https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/

risk_path = "https://www2.census.gov/data/experimental-data-products/community-resilience-estimates/2020/cre-2018-a11.csv"
# If the above breaks, use the downloaded CSV
#risk_path = "data/cre-2018-a11.csv"
risk_df = pd.read_csv(risk_path)

# Break down by North Carolina
nc_risk_df = risk_df.loc[risk_df['STABREV']=='NC']

# Display dataframe
nc_risk_df


,state,county,tract,rfgrp,prednum,prednum_moe,predrt,predrt_moe,stname,STABREV,ctname,popuni
150009,37,1,0,0RF,48284,9880.2,29.84,6.11,North Carolina,NC,"Alamance County, NC",161790
150010,37,1,0,1-2RF,74707,10852.5,46.18,6.71,North Carolina,NC,"Alamance County, NC",161790
150011,37,1,0,3PLRF,38799,9176.3,23.98,5.67,North Carolina,NC,"Alamance County, NC",161790
150012,37,1,20100,0RF,1122,385.8,26.26,9.03,North Carolina,NC,"Alamance County, NC",4272
150013,37,1,20100,1-2RF,2003,410.3,46.89,9.60,North Carolina,NC,"Alamance County, NC",4272
...,...,...,...,...,...,...,...,...,...,...,...,...
156889,37,199,960300,1-2RF,1289,301.9,42.19,9.88,North Carolina,NC,"Yancey County, NC",3055
156890,37,199,960300,3PLRF,827,283.9,27.07,9.29,North Carolina,NC,"Yancey County, NC",3055
156891,37,199,960400,0RF,1135,358.4,26.43,8.34,North Carolina,NC,"Yancey County, NC",4295
156892,37,199,960400,1-2RF,1998,401.6,46.52,9.35,North Carolina,NC,"Yancey County, NC",4295


In [4]:
# -----------------------
# Clean up the data
# -----------------------
nc_risk_df = nc_risk_df[['tract','rfgrp','prednum','ctname','popuni']]
nc_risk_rename_df = nc_risk_df.rename(columns={
    'rfgrp':'risk_group',
    'prednum':'risk_pop',
    'ctname':'county',
    'popuni':'total_pop'})

nc_risk_rename_df['county'].replace(' County, NC','',regex=True,inplace=True)

# We only need to keep the data where tract = 0, as that's the county summaries
nc_risk_grouped = nc_risk_rename_df.groupby(['county','tract','risk_group']).sum()
nc_risk_grouped.reset_index(inplace=True)
nc_risk_filtered = nc_risk_grouped[nc_risk_grouped['tract'] == 0] 

# Display the dataframe
nc_risk_filtered = nc_risk_filtered[['county','risk_group','risk_pop','total_pop']]
nc_risk_filtered

,county,risk_group,risk_pop,total_pop
0,Alamance,0RF,48284,161790
1,Alamance,1-2RF,74707,161790
2,Alamance,3PLRF,38799,161790
111,Alexander,0RF,9789,36096
112,Alexander,1-2RF,17442,36096
...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498
6845,Yadkin,3PLRF,10706,37498
6867,Yancey,0RF,4307,17870
6868,Yancey,1-2RF,8271,17870


In [5]:
# -----------------------
# Report statewide North Carolina risk factor groups
# -----------------------

# Make a new dataframe from the other
nc_overall_risk_df = nc_risk_filtered

# Group by risk factors
nc_overall_risk = nc_overall_risk_df.groupby('risk_group').sum()

# Build new series to hold percentages
nc_overall_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_overall_risk.iterrows():
    nc_overall_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_overall_risk

,risk_pop,total_pop,percentage
risk_group,,,
0RF,2896542,10219101,28.34
1-2RF,4866961,10219101,47.63
3PLRF,2455598,10219101,24.03


In [6]:
# -----------------------
# Report risk factor groups, by county, for North Carolina, based on census data
# -----------------------

nc_county_risk = nc_risk_filtered

# Build new series to hold percentages
nc_county_risk['percentage'] = ''

# Iterate over rows to calculate percentages, round, and store in the new series 
for index, row in nc_county_risk.iterrows():
    nc_county_risk.at[index,'percentage'] = round((row['risk_pop'] / row['total_pop'])*100,2)

# Display the dataframe
nc_county_risk


,county,risk_group,risk_pop,total_pop,percentage
0,Alamance,0RF,48284,161790,29.84
1,Alamance,1-2RF,74707,161790,46.18
2,Alamance,3PLRF,38799,161790,23.98
111,Alexander,0RF,9789,36096,27.12
112,Alexander,1-2RF,17442,36096,48.32
...,...,...,...,...,...
6844,Yadkin,1-2RF,16689,37498,44.51
6845,Yadkin,3PLRF,10706,37498,28.55
6867,Yancey,0RF,4307,17870,24.1
6868,Yancey,1-2RF,8271,17870,46.28


In [7]:
# -----------------------
# This is a NYTimes report on national COVID-19 cases and deaths, sorted by county
# From: https://github.com/nytimes/covid-19-data (us-counties.csv)
# -----------------------

# Pull in the live data
counties_path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
# If the above breaks, we can refer to the saved version of the csv, below:
#counties_path = "data/us-counties.csv"

# Save to a dataframe
covid_df = pd.read_csv(counties_path)

# Sort for NC
covid_county_df = covid_df.loc[covid_df['state']=='North Carolina']

# Drop irrelevant column(s)
covid_county_clean_df = covid_county_df.drop(['fips'], 1)

# Grab last 100 rows (for 100 counties to get latest data)
covid_county_latest_df = covid_county_clean_df.tail(100)

# Diplay the dataframe
covid_county_latest_df

,date,county,state,cases,deaths
352237,2020-07-20,Alamance,North Carolina,1718,39
352238,2020-07-20,Alexander,North Carolina,216,1
352239,2020-07-20,Alleghany,North Carolina,54,0
352240,2020-07-20,Anson,North Carolina,267,1
352241,2020-07-20,Ashe,North Carolina,74,1
...,...,...,...,...,...
352332,2020-07-20,Wayne,North Carolina,2103,34
352333,2020-07-20,Wilkes,North Carolina,663,7
352334,2020-07-20,Wilson,North Carolina,1136,28
352335,2020-07-20,Yadkin,North Carolina,405,6


In [8]:
# -----------------------
# Build a dataframe that shows worst 10 counties in cases and deaths
# -----------------------

# Sort by cases, highest to lowest
sorted_cases = covid_county_latest_df.sort_values(by=['cases'],ascending=False).reset_index()
# Sort by deaths, highest to lowest
sorted_deaths = covid_county_latest_df.sort_values(by=['deaths'],ascending=False).reset_index()

# Build a new dataframe to hold top counties and the value, sorted by cases and deaths
top_df = pd.DataFrame(columns=['Most cases','Case count','Most deaths','Death count'])

# Add the cases to the new dataframe
top_df['Most cases'] = sorted_cases['county'].values
top_df['Case count'] = sorted_cases['cases'].values

# Add the deaths to the new dataframe
top_df['Most deaths'] = sorted_deaths['county'].values
top_df['Death count'] = sorted_deaths['deaths'].values

# Show the top 10 counties for cases and deaths
top_df.head(10)

,Most cases,Case count,Most deaths,Death count
0,Mecklenburg,17571,Mecklenburg,182
1,Wake,8974,Guilford,133
2,Durham,5133,Wake,93
3,Forsyth,4220,Durham,73
4,Guilford,4198,Henderson,52
5,Johnston,2406,Buncombe,50
6,Gaston,2320,Chatham,47
7,Union,2215,Rowan,46
8,Wayne,2103,Robeson,46
9,Cabarrus,1970,Cumberland,46


In [9]:
# -----------------------
# Combine dataframes from NYTimes data and CRE data
# NYT = covid_county_latest_df
# CRE = nc_county_risk
# -----------------------

# Do the merge
combined_data = pd.merge(nc_county_risk, covid_county_latest_df, on='county', how='outer')

# Display the dataframe
combined_data


,county,risk_group,risk_pop,total_pop,percentage,date,state,cases,deaths
0,Alamance,0RF,48284,161790,29.84,2020-07-20,North Carolina,1718,39
1,Alamance,1-2RF,74707,161790,46.18,2020-07-20,North Carolina,1718,39
2,Alamance,3PLRF,38799,161790,23.98,2020-07-20,North Carolina,1718,39
3,Alexander,0RF,9789,36096,27.12,2020-07-20,North Carolina,216,1
4,Alexander,1-2RF,17442,36096,48.32,2020-07-20,North Carolina,216,1
...,...,...,...,...,...,...,...,...,...
295,Yadkin,1-2RF,16689,37498,44.51,2020-07-20,North Carolina,405,6
296,Yadkin,3PLRF,10706,37498,28.55,2020-07-20,North Carolina,405,6
297,Yancey,0RF,4307,17870,24.1,2020-07-20,North Carolina,68,0
298,Yancey,1-2RF,8271,17870,46.28,2020-07-20,North Carolina,68,0


In [10]:
# -----------------------
# Build dataframe to show:
#     Cases and deaths as a percentage of the county population
#     Deaths as a percentage of cases
# -----------------------

# Build empty dataframe and order columns
summary_df = pd.DataFrame(columns={
    'county',
    'cases_pop_pct',
    'deaths_cases_pct',
    'deaths_pop_pct',
    'cases',
    'deaths',
    'population'})
summary_df = summary_df[[
    'county','cases','deaths','population','cases_pop_pct','deaths_pop_pct','deaths_cases_pct']]


# Set empty lists to store values
county_lst = []
cases_lst = []
deaths_lst = []
pop_lst = []

# This variable checks to see if we've seen this county already
county = ''

#Iterate through the data and build lists to go into the new summary dataframe
for i, row in combined_data.iterrows():
    if county == row[0]: # This handles 2nd and 3rd occurrance of county
        pass
    else: # This handles first occurrance of county
        county_lst.append(row[0])
        cases_lst.append(row[7])
        deaths_lst.append(row[8])
        pop_lst.append(row[3])
        
        county = row[0]

# Build the summary dataframe
summary_df['county'] = county_lst
summary_df['cases'] = cases_lst
summary_df['deaths'] = deaths_lst
summary_df['population'] = pop_lst

# Do calculations for percentages
for i, row in summary_df.iterrows():
    summary_df.loc[i,'cases_pop_pct']= f'{round(((row[1]/row[3])*100),4)}%'
    summary_df.loc[i,'deaths_pop_pct']= f'{round(((row[2]/row[3])*100),4)}%'
    summary_df.loc[i,'deaths_cases_pct']= f'{round(((row[2]/row[1])*100),4)}%'

# Display the summary dataframe   
summary_df



,county,cases,deaths,population,cases_pop_pct,deaths_pop_pct,deaths_cases_pct
0,Alamance,1718,39,161790,1.0619%,0.0241%,2.2701%
1,Alexander,216,1,36096,0.5984%,0.0028%,0.463%
2,Alleghany,54,0,11146,0.4845%,0.0%,0.0%
3,Anson,267,1,22949,1.1634%,0.0044%,0.3745%
4,Ashe,74,1,27068,0.2734%,0.0037%,1.3514%
...,...,...,...,...,...,...,...
95,Wayne,2103,34,122647,1.7147%,0.0277%,1.6167%
96,Wilkes,663,7,68118,0.9733%,0.0103%,1.0558%
97,Wilson,1136,28,80922,1.4038%,0.0346%,2.4648%
98,Yadkin,405,6,37498,1.0801%,0.016%,1.4815%
